# Введение в обработку текста на естественном языке

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Введение в обработку текста на естественном языке"
* https://www.nltk.org/api/nltk.metrics.distance.html
* https://pymorphy2.readthedocs.io/en/stable/user/guide.html
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Лабораторная работа 7

1\. Загрузите данные из файла `ru_recipes_sample.csv` в виде `pd.DataFrame` `recipes` Используя регулярные выражения, удалите из описаний (столбец `description`) все символы, кроме русских букв, цифр и пробелов. Приведите все слова в описании к нижнему регистру. Сохраните полученный результат в столбец `description`.

In [47]:
import pandas as pd
import re

In [116]:
recipes = pd.read_csv('ru_recipes_sample.csv')


In [142]:
def find_ru(a):
    p = re.compile(f'[а-яА-Я\s\d]')
    return "".join(p.findall(a)).lower()
    #p = re.findall(f'[а-яА-Я]')
    #return "".join(re.findall(f'[а-яА-Я\s\d]',a))

In [145]:
df['description'] = df['description'].apply(find_ru)

In [146]:
df['description']

0       этот коктейль готовлю из замороженной клубники...
1                                         быстро и вкусно
2                сытный овощной салатик пальчики оближете
3       картофельное пюре и куриные котлеты  вкусная к...
4       вишневая наливка имеет яркий вишневый вкус кот...
                              ...                        
3462    для тех кто любит чечевицу вам сюда очень вкус...
3463    баклажановые фантазии продолжаются предлагаю в...
3464    мое любимое блюдо лазанья но кушать только фар...
3465    прошлым летом варила варенье из одуванчиков по...
3466     и три корочки хлеба  сделал заказ буратино в ...
Name: description, Length: 3467, dtype: object

### Расстояние редактирования

2\. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние Левенштейна. Выведите на экран результат в следующем виде:

```
d(word1, word2) = x
```

In [159]:
from nltk.tokenize import word_tokenize
from nltk import edit_distance

In [604]:
# def word_token(a):
#     return word_tokenize(a)
# word_token('I love you')
unique_words = list(set(sum(list(map(word_tokenize,df['description'])),[])))

In [182]:
import numpy as np

In [193]:
rand_words = np.random.choice(unique_words,10)
for i in range(5):
    rand_1 = np.random.choice(unique_words,1)[0]
    rand_2 = np.random.choice(unique_words,1)[0]
    edit_dst = edit_distance(rand_1,rand_2)
    print(f'd({rand_1},{rand_2}) = {edit_dst}')

d(преследуется,кабачковом) = 12
d(детском,любых) = 7
d(смаченками,шоколаде) = 9
d(меда,малина) = 4
d(добрыми,зубчиков) = 7


3\. Напишите функцию, которая принимает на вход 2 текстовые строки `s1` и `s2` и при помощи расстояния Левенштейна определяет, является ли строка `s2` плагиатом `s1`. Функция должна реализовывать следующую логику: для каждого слова `w1` из `s1` проверяет, есть в `s2` хотя бы одно слово `w2`, такое, что расстояние Левенштейна между `w1` и `w2` меньше 2, и считает количество таких слов в `s1` $P$. 

$$ P = \#\{w_1 \in s_1\ | \exists w_2 \in s_2 : d(w_1, w_2) < tol\}$$

$$ L = max(|s1|, |s2|) $$

Здесь $|\cdot|$ - количество слов в строке, $\#A$ - число элементов в множестве $A$, $w \in s$ означает, что слово $w$ содержится в тексте $s$.

Если отношение $P / L$ больше 0.8, то функция должна вернуть True; иначе False.

Продемонстрируйте работу вашей функции на примере описаний двух рецептов с ID 135488 и 851934 (ID рецепта - это число, стоящее в конце url рецепта). Выведите на экран описания этих рецептов и результат работы функции.

In [619]:
def func_re(a):
    p = re.compile(r'851934|135488')
    if p.findall(str(a)) != []:
        return  1 #p.findall(str(a))
    else:
        return None

In [620]:
pd.DataFrame(df['url'].apply(func_re)).isna().sum()

url    3465
dtype: int64

In [611]:
def is_plagiarism(s1: str, s2: str) -> bool:
    P = 0
    L = max(len(word_tokenize(s1)),len(word_tokenize(s2)))
    for w1 in word_tokenize(s1):
        for w2 in word_tokenize(s2):
            if edit_distance(w1,w2) < 2:
                P += 1 #s1.count(j)
                #print(w1,w2)
                break
                #print(s1.count(j),j)
    if P/L > 0.8:
        return True
    else:
        return False


In [612]:
is_plagiarism(s1,s2)

True

In [613]:
# [i for i in word_tokenize(s1)]
# [i for i in word_tokenize(s2)]

In [614]:
# print(word_tokenize(s1))

In [615]:
# print(word_tokenize(s2))

In [617]:
# s1 = df.loc[1473][3]#df['description'][2]
# s2 = df.loc[958][3]#df['description'][5]
# P = 0
# L = 0
# for i in word_tokenize(s1):
#     for j in word_tokenize(s2):
#         if edit_distance(i,j) < 2:
#             #print(s1.count(i),i)
#             P = s1.count(i)
#             L = max(len(word_tokenize(s1)),len(word_tokenize(s2)))
# P/L

### Стемминг, лемматизация

4\. На основе набора слов из задания 2 создайте `pd.DataFrame` со столбцами `word`, `stemmed_word` и `normalized_word`. В столбец `stemmed_word` поместите версию слова после проведения процедуры стемминга; в столбец `normalized_word` поместите версию слова после проведения процедуры лемматизации. Столбец `word` укажите в качестве индекса. 

Для стемминга можно воспользоваться `SnowballStemmer` из `nltk`, для лемматизации слов - пакетом `pymorphy2`. Сравните результаты стемминга и лемматизации. Поясните на примере одной из строк получившегося фрейма (в виде текстового комментария), в чем разница между двумя этими подходами. 

In [415]:
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
import pymorphy2

In [416]:
stemmer = SnowballStemmer("russian")

In [417]:
morph = pymorphy2.MorphAnalyzer()

In [434]:
normalized_word = [morph.parse(w)[0].normalized.word for w in unique_words]

In [425]:
stemmed_word = [stemmer.stem(w) for w in unique_words]

In [442]:
df_1 = pd.DataFrame(stemmed_word, columns=['stemmed_word'],index = unique_words)
df_1['normalized_word'] = normalized_word

In [443]:
df_1.head()

,stemmed_word,normalized_word
кексы,кекс,кекс
сметаны,смета,сметана
соседей,сосед,сосед
ансамбль,ансамбл,ансамбль
нужная,нужн,нужный


5\. Добавьте в таблицу `recipes` столбец `description_no_stopwords`, в котором содержится текст описания рецепта после удаления из него стоп-слов. Посчитайте и выведите на экран долю стоп-слов среди общего количества слов. Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [531]:
import nltk

In [484]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Payrav\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [488]:
new_stopwords = stopwords.words('russian')


In [530]:
def desc_stop_words(a):
    return [word for word in word_tokenize(str(a)) if word not in new_stopwords]

In [527]:
recipes['description_no_stopwords'] = df['description'].apply(desc_stop_words)

In [602]:
unique_words_1 = len(list(sum(list(map(word_tokenize,df['description'])),[]))) #Кол-во слов без удаления стоп слов

In [587]:
m = sum(list(map(len, recipes['description_no_stopwords']))) #Кол-во слов после удаления стоп слов

In [603]:
print(f'{(unique_words_1 - m) * 100/ unique_words} %')

32.267086173745845 %


In [673]:
l = np.array(sum(list(map(word_tokenize, df['description'])),[]))

In [672]:
uniq, counts = np.unique(l, return_counts= True)
top_words= dict(sorted(dict(zip(uniq,counts)).items(), key=lambda x: x[1], reverse=True)) #top 10 до
list(top_words.keys())[:10] #top 10 до

['и', 'в', 'с', 'на', 'очень', 'не', 'из', 'я', 'рецепт', 'а']

In [671]:
# l_2 = np.array(sum(list(map(word_tokenize, recipes['description_no_stopwords'])),[]))
# uniq, counts = np.unique(l_2, return_counts= True)
# # sorted(counts,reverse=True)
# top_words= dict(sorted(dict(zip(uniq,counts)).items(), key=lambda x: x[1], reverse=True)) #top 10 до
# list(top_words.keys())[:10] #top 10 до

In [ ]:
# recipes['description_no_stopwords'].apply(length).sum()

In [ ]:
# def length(b):
#     return len(b)


### Векторное представление текста

6\. Выберите случайным образом 5 рецептов из набора данных, в названии которых есть слово "оладьи" (без учета регистра). Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`. На основе полученных векторов создайте `pd.DataFrame`, в котором названия колонок соответствуют словам из словаря объекта-векторизатора. 

Примечание: обратите внимание на порядок слов при создании колонок.

In [702]:
def find_ru(a):
    p = re.compile(f'Куриные')
    return p.findall(str(a))

In [703]:
find_ru(df['name'])


['Куриные']

In [682]:
df['name']

0                      Густой молочно-клубничный коктейль
1                                                Рулетики
2                                     Салат "Баклажанчик"
3           Куриные котлеты с картофельным пюре в духовке
4                                 Рецепт вишневой наливки
                              ...                        
3462                                                 Мшош
3463                     Мясные треугольники с баклажаном
3464                                 "Болоньез" по-новому
3465                 Варенье из одуванчиков с апельсинами
3466    Три корочки хлеба под соусом болоньез для Пино...
Name: name, Length: 3467, dtype: object

7\. Вычислите близость между каждой парой рецептов, выбранных в задании 6, используя косинусное расстояние (можно воспользоваться функциями из любого пакета: `scipy`, `scikit-learn` или реализовать функцию самому). Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

Примечание: обратите внимание, что $d_{cosine}(x, x) = 0$

8\. Напишите функцию, которая принимает на вход `pd.DataFrame`, полученный в задании 7, и возвращает в виде кортежа названия двух различных рецептов, которые являются наиболее похожими. Прокомментируйте результат (в виде текстового комментария). Для объяснения результата сравните слова в описаниях двух этих отзывов.

In [ ]:
def find_closest(sim_df: pd.DataFrame) -> tuple:
    pass